In [1]:
import glob
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import operator 
from collections import Counter
from nltk import bigrams
import pandas as pd
import csv
import unicodecsv
from os import listdir
from os.path import isfile, join
import glob
import wordcloud
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = "full_DataFrame/combined_data/users_data/"
onlyfiles = [path+f for f in listdir(path) if isfile(join(path, f))]
onlyfiles

['full_DataFrame/combined_data/users_data/id_time_fCount_f_retweet_count_user_1.msg',
 'full_DataFrame/combined_data/users_data/id_time_fCount_f_retweet_count_user_2.msg',
 'full_DataFrame/combined_data/users_data/id_time_fCount_f_retweet_count_user_3.msg',
 'full_DataFrame/combined_data/users_data/id_time_fCount_f_retweet_count_user_4.msg',
 'full_DataFrame/combined_data/users_data/id_time_fCount_f_retweet_count_user_5.msg']

In [3]:
df = pd.DataFrame()
for i in range(0, len(onlyfiles)):
    dfA = pd.read_msgpack(onlyfiles[i])
    df = df.append(dfA, ignore_index=True)
df.head()

id                      created_at  favorite_count favorited  \
0  9.635508e+17  Tue Feb 13 23:09:37 +0000 2018             0.0     False   
1  9.635508e+17  Tue Feb 13 23:09:38 +0000 2018             0.0     False   
2  9.635508e+17  Tue Feb 13 23:09:38 +0000 2018             0.0     False   
3  9.635508e+17  Tue Feb 13 23:09:38 +0000 2018             0.0     False   
4  9.635508e+17  Tue Feb 13 23:09:38 +0000 2018             0.0     False   

   retweet_count             user_id                 user_created_at  \
0            0.0  903604997564325888  Fri Sep 01 13:06:34 +0000 2017   
1            0.0  903683024407207936  Fri Sep 01 18:16:37 +0000 2017   
2            0.0          2707415359  Mon Aug 04 20:31:07 +0000 2014   
3            0.0  962627348943302656  Sun Feb 11 10:00:19 +0000 2018   
4            0.0  953593151922487296  Wed Jan 17 11:41:39 +0000 2018   

   user_followers_count  user_friends_count user_location user_screen_name  \
0                     0                   0          None  NYs1YM9lziqMuwR   
1                     0                   0          None  HEOYvB8PVGLO1ux   
2                   890                  79          None      Epimetheusz   
3                     0                  76  Thane, India  gabrielleskitt5   
4                   146                 409    Texas, USA       btc_monkey   

   user_verified                        user_description  
0          False                                    None  
1          False                                    None  
2          False  Crypto addict since 2013. I need help.  
3          False                                    None  
4          False                                    None

In [4]:
df.shape

(7822648, 13)

In [5]:
df = df.drop_duplicates(subset=['id'], keep=False)

In [6]:
df.shape

(6406528, 13)

In [7]:
# converting to date to pd date time
df['created_at_time'] = pd.to_datetime(df.created_at)

In [12]:
df['created_at'] = df['created_at_time']

In [13]:
df.head()

id          created_at  favorite_count favorited  retweet_count  \
0  9.635508e+17 2018-02-13 23:09:37             0.0     False            0.0   
1  9.635508e+17 2018-02-13 23:09:38             0.0     False            0.0   
2  9.635508e+17 2018-02-13 23:09:38             0.0     False            0.0   
3  9.635508e+17 2018-02-13 23:09:38             0.0     False            0.0   
4  9.635508e+17 2018-02-13 23:09:38             0.0     False            0.0   

              user_id                 user_created_at  user_followers_count  \
0  903604997564325888  Fri Sep 01 13:06:34 +0000 2017                     0   
1  903683024407207936  Fri Sep 01 18:16:37 +0000 2017                     0   
2          2707415359  Mon Aug 04 20:31:07 +0000 2014                   890   
3  962627348943302656  Sun Feb 11 10:00:19 +0000 2018                     0   
4  953593151922487296  Wed Jan 17 11:41:39 +0000 2018                   146   

   user_friends_count user_location user_screen_name  user_verified  \
0                   0          None  NYs1YM9lziqMuwR          False   
1                   0          None  HEOYvB8PVGLO1ux          False   
2                  79          None      Epimetheusz          False   
3                  76  Thane, India  gabrielleskitt5          False   
4                 409    Texas, USA       btc_monkey          False   

                         user_description     created_at_time  
0                                    None 2018-02-13 23:09:37  
1                                    None 2018-02-13 23:09:38  
2  Crypto addict since 2013. I need help. 2018-02-13 23:09:38  
3                                    None 2018-02-13 23:09:38  
4                                    None 2018-02-13 23:09:38

In [14]:
print(df.created_at.min())
print(df.created_at.max())

2018-02-13 23:09:37
2018-03-03 08:42:41


In [15]:
((df['created_at'][0] - pd.to_datetime(df['user_created_at'][0]) 
  + df['created_at'][0] - pd.to_datetime(df['user_created_at'][0]))/2).days

165

In [16]:
mask = (df['created_at'] > '2018-02-27 05:05:02') & (df['created_at'] <= '2018-03-03 08:43:41')
masked = df.loc[mask]

In [17]:
masked.head()

id          created_at  favorite_count favorited  \
3680051  9.683515e+17 2018-02-27 05:06:02             0.0     False   
3680052  9.683515e+17 2018-02-27 05:06:03             0.0     False   
3680053  9.683515e+17 2018-02-27 05:06:03             0.0     False   
3680054  9.683515e+17 2018-02-27 05:06:04             0.0     False   
3680055  9.683515e+17 2018-02-27 05:06:04             0.0     False   

         retweet_count             user_id                 user_created_at  \
3680051            0.0           765556298  Sat Aug 18 11:37:24 +0000 2012   
3680052            0.0          3397936720  Fri Jul 31 23:11:15 +0000 2015   
3680053            0.0          1712283061  Fri Aug 30 09:24:00 +0000 2013   
3680054            0.0  963322636980948993  Tue Feb 13 08:03:09 +0000 2018   
3680055            0.0          2957304731  Fri Jan 02 21:05:49 +0000 2015   

         user_followers_count  user_friends_count  user_location  \
3680051                   328                  60           None   
3680052                 16351                 132       Việt Nam   
3680053                  1646                4048         Owerri   
3680054                     6                  92           None   
3680055                   672                  15  International   

        user_screen_name  user_verified  \
3680051     bitcoinnews9          False   
3680052       badoyan932          False   
3680053    AnthonyEzinwa          False   
3680054       Vidoor1302          False   
3680055    cryptograbber          False   

                                          user_description     created_at_time  
3680051                    #bitcoin #bitcoins #bitcoinnews 2018-02-27 05:06:02  
3680052  🎯ПАРТНЕР GOLDEN ISLAND.\n.\nУЧУ ЗАРАБАТЫВАТЬ Н... 2018-02-27 05:06:03  
3680053         0x3cA0F0214FD32235f54A14141a5d5ef5b89290B3 2018-02-27 05:06:03  
3680054                                               None 2018-02-27 05:06:04  
3680055                            Cryptotize your Life... 2018-02-27 05:06:04

In [18]:
masked.shape

(2067946, 14)

In [19]:
masked.to_msgpack("full_DataFrame/combined_data/master/data_users_min.msg")

In [20]:
# converting to date to pd date time
masked.user_created_at = pd.to_datetime(masked.user_created_at)

C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [37]:
masked.to_msgpack("full_DataFrame/combined_data/master/masked_data_users_min.msg")

In [24]:
# masked['user_age'] = np.where(masked['id'], 0, 0)
masked['user_age'] = masked['created_at'] - pd.to_datetime(masked['user_created_at'])

C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
masked['user_verified'] = masked['user_verified']*1
masked['user_followers_count'] = masked['user_followers_count']*1
masked['user_friends_count'] = masked['user_friends_count']*1
# masked['user_verified'] = masked['user_verified']*1
# masked['user_verified'] = masked['user_verified']*1

C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [29]:
masked['user_age_days'] = [ele.days for ele in masked['user_age']]
# masked["RT"] = [1 if "RT" in ele else 0 for ele in masked["text"]]
masked.head()

C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


id          created_at  favorite_count favorited  \
3680051  9.683515e+17 2018-02-27 05:06:02             0.0     False   
3680052  9.683515e+17 2018-02-27 05:06:03             0.0     False   
3680053  9.683515e+17 2018-02-27 05:06:03             0.0     False   
3680054  9.683515e+17 2018-02-27 05:06:04             0.0     False   
3680055  9.683515e+17 2018-02-27 05:06:04             0.0     False   

         retweet_count             user_id     user_created_at  \
3680051            0.0           765556298 2012-08-18 11:37:24   
3680052            0.0          3397936720 2015-07-31 23:11:15   
3680053            0.0          1712283061 2013-08-30 09:24:00   
3680054            0.0  963322636980948993 2018-02-13 08:03:09   
3680055            0.0          2957304731 2015-01-02 21:05:49   

         user_followers_count  user_friends_count  user_location  \
3680051                   328                  60           None   
3680052                 16351                 132       Việt Nam   
3680053                  1646                4048         Owerri   
3680054                     6                  92           None   
3680055                   672                  15  International   

        user_screen_name  user_verified  \
3680051     bitcoinnews9              0   
3680052       badoyan932              0   
3680053    AnthonyEzinwa              0   
3680054       Vidoor1302              0   
3680055    cryptograbber              0   

                                          user_description  \
3680051                    #bitcoin #bitcoins #bitcoinnews   
3680052  🎯ПАРТНЕР GOLDEN ISLAND.\n.\nУЧУ ЗАРАБАТЫВАТЬ Н...   
3680053         0x3cA0F0214FD32235f54A14141a5d5ef5b89290B3   
3680054                                               None   
3680055                            Cryptotize your Life...   

            created_at_time           user_age  user_age_days  
3680051 2018-02-27 05:06:02 2018 days 17:28:38           2018  
3680052 2018-02-27 05:06:03  941 days 05:54:48            941  
3680053 2018-02-27 05:06:03 1641 days 19:42:03           1641  
3680054 2018-02-27 05:06:04   13 days 21:02:55             13  
3680055 2018-02-27 05:06:04 1151 days 08:00:15           1151

In [38]:
masked = masked.drop_duplicates(subset=['user_id'], keep='first')

In [39]:
# masked = masked.reset_index()

In [40]:
# for i in range(0, len(masked)):
#     masked['user_age'][i] = (masked['created_at'][i] - pd.to_datetime(masked['user_created_at'][i])).days

In [41]:
masked.shape

(365391, 16)

In [42]:
masked.columns

Index(['created_at', 'id', 'favorite_count', 'favorited', 'retweet_count',
       'user_id', 'user_created_at', 'user_followers_count',
       'user_friends_count', 'user_location', 'user_screen_name',
       'user_verified', 'user_description', 'created_at_time', 'user_age',
       'user_age_days'],
      dtype='object')

In [44]:
masked_df = masked[['created_at','user_followers_count', 'user_friends_count', 'user_verified', 'user_age_days']]
masked_df.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:06:02                   328                  60   
1 2018-02-27 05:06:03                 16351                 132   
2 2018-02-27 05:06:03                  1646                4048   
3 2018-02-27 05:06:04                     6                  92   
4 2018-02-27 05:06:04                   672                  15   

   user_verified  user_age_days  
0              0           2018  
1              0            941  
2              0           1641  
3              0             13  
4              0           1151

In [45]:
masked_df['no_users']=1
masked_df.head()

C:\Users\mkdv6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:06:02                   328                  60   
1 2018-02-27 05:06:03                 16351                 132   
2 2018-02-27 05:06:03                  1646                4048   
3 2018-02-27 05:06:04                     6                  92   
4 2018-02-27 05:06:04                   672                  15   

   user_verified  user_age_days  no_users  
0              0           2018         1  
1              0            941         1  
2              0           1641         1  
3              0             13         1  
4              0           1151         1

In [51]:
masked_df.set_index('created_at', inplace=True)
masked_df.head()

user_followers_count  user_friends_count  user_verified  \
created_at                                                                     
2018-02-27 05:06:02                   328                  60              0   
2018-02-27 05:06:03                 16351                 132              0   
2018-02-27 05:06:03                  1646                4048              0   
2018-02-27 05:06:04                     6                  92              0   
2018-02-27 05:06:04                   672                  15              0   

                     user_age_days  no_users  
created_at                                    
2018-02-27 05:06:02           2018         1  
2018-02-27 05:06:03            941         1  
2018-02-27 05:06:03           1641         1  
2018-02-27 05:06:04             13         1  
2018-02-27 05:06:04           1151         1

In [52]:
users_data_2min = masked_df.groupby(pd.TimeGrouper('2Min')).sum()
print(users_data_2min.shape)
users_data_2min.head()

(2989, 5)


user_followers_count  user_friends_count  user_verified  \
created_at                                                                     
2018-02-27 05:06:00             1864130.0            632256.0            3.0   
2018-02-27 05:08:00             3048409.0           1050352.0            3.0   
2018-02-27 05:10:00             1307182.0            693767.0            1.0   
2018-02-27 05:12:00              741846.0            562689.0            3.0   
2018-02-27 05:14:00             2056162.0           1721122.0            1.0   

                     user_age_days  no_users  
created_at                                    
2018-02-27 05:06:00       450567.0     422.0  
2018-02-27 05:08:00       414776.0     381.0  
2018-02-27 05:10:00       401555.0     376.0  
2018-02-27 05:12:00       323445.0     346.0  
2018-02-27 05:14:00       353773.0     340.0

In [53]:
users_data_2min = users_data_2min.reset_index()
users_data_2min.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:06:00             1864130.0            632256.0   
1 2018-02-27 05:08:00             3048409.0           1050352.0   
2 2018-02-27 05:10:00             1307182.0            693767.0   
3 2018-02-27 05:12:00              741846.0            562689.0   
4 2018-02-27 05:14:00             2056162.0           1721122.0   

   user_verified  user_age_days  no_users  
0            3.0       450567.0     422.0  
1            3.0       414776.0     381.0  
2            1.0       401555.0     376.0  
3            3.0       323445.0     346.0  
4            1.0       353773.0     340.0

In [57]:
users_data_2min['avg_id_age_days'] = users_data_2min['user_age_days']/users_data_2min['no_users']
users_data_2min.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:06:00             1864130.0            632256.0   
1 2018-02-27 05:08:00             3048409.0           1050352.0   
2 2018-02-27 05:10:00             1307182.0            693767.0   
3 2018-02-27 05:12:00              741846.0            562689.0   
4 2018-02-27 05:14:00             2056162.0           1721122.0   

   user_verified  user_age_days  no_users   avg_id_age  avg_id_age_days  
0            3.0       450567.0     422.0  1067.694313      1067.694313  
1            3.0       414776.0     381.0  1088.650919      1088.650919  
2            1.0       401555.0     376.0  1067.965426      1067.965426  
3            3.0       323445.0     346.0   934.812139       934.812139  
4            1.0       353773.0     340.0  1040.508824      1040.508824

In [59]:
users_data_2min = users_data_2min.drop(['user_age_days', 'avg_id_age'],axis=1)
users_data_2min.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:06:00             1864130.0            632256.0   
1 2018-02-27 05:08:00             3048409.0           1050352.0   
2 2018-02-27 05:10:00             1307182.0            693767.0   
3 2018-02-27 05:12:00              741846.0            562689.0   
4 2018-02-27 05:14:00             2056162.0           1721122.0   

   user_verified  no_users  avg_id_age_days  
0            3.0     422.0      1067.694313  
1            3.0     381.0      1088.650919  
2            1.0     376.0      1067.965426  
3            3.0     346.0       934.812139  
4            1.0     340.0      1040.508824

In [60]:
users_data_2min.to_msgpack("full_DataFrame/combined_data/master/users_data_2min.msg")

In [61]:
users_data_5min = masked_df.groupby(pd.TimeGrouper('5Min')).sum()
print(users_data_5min.shape)
users_data_5min.head()

(1196, 5)


user_followers_count  user_friends_count  user_verified  \
created_at                                                                     
2018-02-27 05:05:00             4912539.0           1682608.0            6.0   
2018-02-27 05:10:00             3692989.0           2775444.0            5.0   
2018-02-27 05:15:00             1877669.0           1015009.0            3.0   
2018-02-27 05:20:00             2333556.0           1206450.0            4.0   
2018-02-27 05:25:00             1816883.0           1065921.0            3.0   

                     user_age_days  no_users  
created_at                                    
2018-02-27 05:05:00       865343.0     803.0  
2018-02-27 05:10:00       922915.0     917.0  
2018-02-27 05:15:00       829870.0     772.0  
2018-02-27 05:20:00       770209.0     718.0  
2018-02-27 05:25:00       737464.0     628.0

In [62]:
users_data_5min = users_data_5min.reset_index()
users_data_5min.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:05:00             4912539.0           1682608.0   
1 2018-02-27 05:10:00             3692989.0           2775444.0   
2 2018-02-27 05:15:00             1877669.0           1015009.0   
3 2018-02-27 05:20:00             2333556.0           1206450.0   
4 2018-02-27 05:25:00             1816883.0           1065921.0   

   user_verified  user_age_days  no_users  
0            6.0       865343.0     803.0  
1            5.0       922915.0     917.0  
2            3.0       829870.0     772.0  
3            4.0       770209.0     718.0  
4            3.0       737464.0     628.0

In [64]:
users_data_5min['avg_id_age_days'] = users_data_5min['user_age_days']/users_data_5min['no_users']
users_data_5min = users_data_5min.drop(['user_age_days'],axis=1)
users_data_5min.head()

created_at  user_followers_count  user_friends_count  \
0 2018-02-27 05:05:00             4912539.0           1682608.0   
1 2018-02-27 05:10:00             3692989.0           2775444.0   
2 2018-02-27 05:15:00             1877669.0           1015009.0   
3 2018-02-27 05:20:00             2333556.0           1206450.0   
4 2018-02-27 05:25:00             1816883.0           1065921.0   

   user_verified  no_users  avg_id_age_days  
0            6.0     803.0      1077.637609  
1            5.0     917.0      1006.450382  
2            3.0     772.0      1074.961140  
3            4.0     718.0      1072.714485  
4            3.0     628.0      1174.305732

In [65]:
users_data_5min.to_msgpack("full_DataFrame/combined_data/master/users_data_5min.msg")

In [67]:
users_data_5min.columns

Index(['created_at', 'user_followers_count', 'user_friends_count',
       'user_verified', 'no_users', 'avg_id_age_days'],
      dtype='object')